# Checking derivatives

## Finite differencing with Numpy

In [ ]:
import numpy as np

In [ ]:
def finite_diff(f, x, delta):
    """estimate the gradient by finite-differencing method"""
    grad_f, dx = np.zeros_like(x), np.zeros_like(x)
    for i in range(x.size):
        dx.flat[i]     = delta
        grad_f.flat[i] = f(x+dx) - f(x-dx)
        dx.flat[i]     = 0.0
    return grad_f / (2*delta)

In [ ]:
# example 1: least squares
m, n = 3,4
y, A = np.random.randn(m,1), np.random.randn(m,n)
phi1      = lambda x: (y-A@x).T @ (y-A@x)
grad_phi1 = lambda x: -2*A.T@(y-A@x)  # derived by hand
x  = np.random.randn(n, 1)
g1 = grad_phi1(x)
g2 = finite_diff(phi1, x, 1e-5)
print(g1)
print(g2)
print(np.linalg.norm(g1-g2))

In [ ]:
# example 2: scalar-valued function of a matrix
m, n = 4,3
y, c = np.random.randn(m, 1), np.random.randn(n, 1)
phi2 = lambda X: c.T @ np.linalg.inv(X.T @ X) @ X.T @ y
def grad_phi2(X):
    A = np.linalg.inv(X.T @ X)
    return -X@A@X.T@y@c.T@A -X@A@c@y.T@X@A +y@c.T@A
X = np.random.randn(m, n)
g1 = grad_phi2(X)
g2 = finite_diff(phi2, X, 1e-5)
print(g1)
print(g2)
print(np.linalg.norm(g1-g2))

In [ ]:
# example 3: Rayleigh coefficient
m = 4
A = np.random.randn(m, m)
A = A + A.T   # symmetrize 
phi3 = lambda x: (x.T @ A @ x) / (x.T @ x)
def grad_phi3(x):
    xx = x.T@x
    return 2*(A@x)/xx - 2*(x.T@A@x)/(xx*xx) * x
x = np.random.randn(m, 1)
g1 = grad_phi3(x)
g2 = finite_diff(phi3, x, 1e-5)
print(g1)
print(g2)
print(np.linalg.norm(g1-g2))

## Automatic differentiation with pytorch

In [ ]:
import torch

In [ ]:
def torch_gradient(f, x):
    y = f(x)
    y.backward()
    return x.grad

In [ ]:
# example 1: least squares
m, n = 3,4
y, A = torch.randn(m,1), torch.randn(m,n)
phi1      = lambda x: (y-A@x).T @ (y-A@x)
grad_phi1 = lambda x: -2*A.T@(y-A@x)  # derived by hand
x = torch.randn(n,1, requires_grad=True)
g1 = grad_phi1(x)
g2 = torch_gradient(phi1, x)
print(g1)
print(g2)
print(torch.norm(g1-g2))

In [ ]:
# example 2: scalar-valued function of a matrix
m, n = 4,3
y, c = torch.randn(m, 1), torch.randn(n, 1)
phi2 = lambda X: c.T @ torch.inverse(X.T @ X) @ X.T @ y
def grad_phi2(X):
    A = torch.inverse(X.T @ X)
    return -X@A@X.T@y@c.T@A -X@A@c@y.T@X@A +y@c.T@A
X = torch.randn(m, n, requires_grad=True)
g1 = grad_phi2(X)
g2 = torch_gradient(phi2, X)
print(g1)
print(g2)
print(torch.norm(g1-g2))

In [ ]:
# example 3: Rayleigh coefficient
m = 4
A = torch.randn(m, m)
A = A + A.T   # symmetrize 
phi3 = lambda x: (x.T @ A @ x) / (x.T @ x)
def grad_phi3(x):
    xx = x.T@x
    return 2*(A@x)/xx - 2*(x.T@A@x)/(xx*xx) * x
x = torch.randn(m, 1, requires_grad=True)
g1 = grad_phi3(x)
g2 = torch_gradient(phi3, x)
print(g1)
print(g2)
print(torch.norm(g1-g2))